In [1]:
print(sc.version)

2.1.0


In [33]:
# Import required libraries

# Regex
import re

#import nltk
from nltk.corpus import stopwords

from nltk.stem.porter import *
stemmer = PorterStemmer()

# NetworkX graph library
import networkx as nx

# Random library
import random

# File libraries
import os
import glob
from pathlib import Path

# Time Library
from time import time

# Maths library
import math

# Scipy stats functions
import scipy.stats
import numpy as np

# KD Tree algo
from sklearn.neighbors import KDTree

# kmeans cluster algorithm 
from sklearn.cluster import KMeans

# Confusion matrix, precision, recall, F1
from sklearn.metrics import *

# Pandas library
import pandas as pd

# Operator
import operator

In [29]:
# Get list of Google stopwords from file

fname = "C:\\Users\\Stephen\\OneDrive\\Documents\\NCI\\Thesis\\Data\\GoogleStopwords.txt"

with open(fname) as f:
    StopWords = f.readlines()
# remove whitespace characters like `\n` at the end of each line
StopWords = [x.strip() for x in StopWords]
# Stem the stopwords
StopWords = list(set([stemmer.stem(y) for y in StopWords]))



In [3]:
# Create list of stop words to use- first the NLTK stopwords
cachedStopWords = stopwords.words("english")
# Add in the Google Stopwords
cachedStopWords += StopWords
#print(cachedStopWords)

In [4]:
# VB style text parsing functions
def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

def mid(s, offset, amount):
    return s[offset:offset+amount]


In [5]:
# This function cleans a piece of text of non letter/space characters
p1 = re.compile(r'[^a-z ]', re.UNICODE)
p2 = re.compile(r' +', re.UNICODE)

def CleanWord(w):
    x = w.lower()
    #x = re.sub(r'[^a-z ]','',x)
    x = p1.sub(' ', x) #.strip() 
    x = p2.sub(' ', x).strip()
    #x = x.split(' ')
    return x

print(CleanWord("the quick Quick \r\nbrown.\r\nfox 1.2)"))

the quick quick brown fox


In [6]:
import csv

def StoreGraphToFile(Graph, PassNum, Stage, Desc):
    passNo = right('00' + str(PassNum), 2)
    filename = 'C:\\BBC_Data\\bbc-sportext\\Pass' + passNo + '.' + str(Stage) + ' ' + Desc + '.csv'
    myfile = open(filename, 'w')
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

    for n,d in Graph.nodes_iter(data=True):
        mylist=str(n) + "\t" + str(d["type"]) + "\t" + str(d["x"]) + "\t" + str(d["y"]) + "\n"
        myfile.write(mylist)

    myfile.close()

In [8]:
# Window size to use for graph-of-words extraction
windowSize = 3


# Function to parse out words and bigrams
def FileToGraph(fileTuple):
    # Array to return from function
    rArr = []

    # Take the filename from the tuple
    filename = fileTuple[0]
    
    # remove non letters from the text in the tuple
    r = CleanWord(fileTuple[1])
    
    # split into an array, stem, and remove words in the stopword list
    arr = [stemmer.stem(y) for y in r.split(" ")]
    arr = [word for word in arr if word not in cachedStopWords]
    
    # Create a new directed graph
    G = nx.DiGraph()
    
    arrLen = len(arr)

    # Add the first few nodes
    for i in range(0, windowSize-1):
        G.add_node(arr[i])

    for i in range(windowSize-1, arrLen):
        G.add_node(arr[i])
        for j in range(0, (windowSize-1)):
            src = arr[i-(windowSize-1)]
            tgt = arr[i-j]
            if G.has_edge(src, tgt):
                # we added this one before, just increase the weight by one
                G[src][tgt]['weight'] += 1
            else:
                # new edge. add with weight=1
                G.add_edge(src, tgt, weight=1)

    for i in range(arrLen-(windowSize-1), arrLen):
        for j in range(1, (arrLen-i)):
            src = arr[i]
            tgt = arr[i+j]
            if G.has_edge(src, tgt):
                # we added this one before, just increase the weight by one
                G[src][tgt]['weight'] += 1
            else:
                # new edge. add with weight=1
                G.add_edge(src, tgt, weight=1)
        
    # Get the "indegree" of the terms
    #d = G.degree()
    d = G.in_degree()

    # Now add all the words the the return array with the filename and the count
    for term in d:
        rArr.append(filename + "\t" + term + "\t" + str(d[term]))
        
    # Return the array of graphs and the array of words
    return rArr
    #return G

text = 'in natural language processing (NLP) a text graph is a graph representation of a text item '
text += '(document, passage or sentence) it is typically created as a preprocessing step to support '
text += 'NLP tasks such as text condensation term disambiguation (topic based) text summarization '
text += '(summarize large text collections) and relation extraction (extract relations from unstructured text)'
d = ['file://file/name.ext', text]

x = FileToGraph(d)
for y in x: #.edges():
    print(y)

file://file/name.ext	step	2
file://file/name.ext	larg	1
file://file/name.ext	disambigu	2
file://file/name.ext	creat	2
file://file/name.ext	nlp	4
file://file/name.ext	topic	2
file://file/name.ext	collect	2
file://file/name.ext	natur	0
file://file/name.ext	typic	2
file://file/name.ext	document	2
file://file/name.ext	preprocess	2
file://file/name.ext	extract	3
file://file/name.ext	represent	1
file://file/name.ext	sentenc	2
file://file/name.ext	task	2
file://file/name.ext	item	2
file://file/name.ext	condens	2
file://file/name.ext	term	2
file://file/name.ext	unstructur	2
file://file/name.ext	process	2
file://file/name.ext	summar	3
file://file/name.ext	graph	3
file://file/name.ext	relat	3
file://file/name.ext	support	2
file://file/name.ext	languag	1
file://file/name.ext	text	11
file://file/name.ext	passag	2
file://file/name.ext	base	2


In [9]:
# Get all the files - one file at a time
t0 = time()

# LOCAL PYTHON IMPLEMENTATION:
def GraphTextFiles(p):
    # If this is a folder, we need to append *.* for glob
    if(os.path.isdir(p)==True):
        p+="\\*.*"
    
    f_list = glob.glob(p)
    rVal = []
    testSet = []
    
    for f in f_list:
        contents = Path(f).read_text()
        # Hive off a random 10% for a test set
        if random.random() < .1:
            testSet += FileToGraph((f, contents))
        else:
            rVal += FileToGraph((f, contents))
        
    return rVal, testSet

text_files, test_set = GraphTextFiles("C:\\BBC_Data\\bbc-sportext\\bbcAll\\????.txt")

## Write initial files and term weights to file
#filename = 'C:\\BBC_Data\\bbc-fulltext\\Initial_Graph_Text.csv'
#myfile = open(filename, 'w')
#wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#
#for x in text_files:
#    myfile.write(x + "\n")
#
#myfile.close()

print("done in %0.3fs." % (time() - t0))

# PYSPARK IMPLEMENTATION: 
#text_files = sc.wholeTextFiles("C:\\BBC_Data\\bbc-fulltext\\bbcAll\\*.txt") \
#    .flatMap(lambda fileTuple: FileToGraph(fileTuple))
#text_files.saveAsTextFile("C:\\BBC_Data\\Output\\files")



done in 24.203s.


In [10]:
print(len(test_set)/len(text_files) * 100)

13.260207373617632


In [11]:
# Create the initial graph
Gr=nx.Graph()
GTest=nx.Graph()

# fArr will hold the array of filenames
fArr = []
fTest = []

# tArr will hold the array of terms added to the graph
tArr = []
tTest = []

# tArr1 will hold the array of terms that we have seen at least once
tArr1 = []

t0 = time()

for x in text_files:   # FOR SPARK APPEND: .collect():

    # Split out the 
    y = x.split("\t")
    filename = y[0]
    term = y[1]
    degree = y[2]
    label = filename[32:33]
    
    # If the filename is not already added, add it to the graph
    if not filename in fArr:

        # Create some random spatial positions
        xR = random.random()*1000
        yR = random.random()*1000
        
        # Add the file node to the graph
        Gr.add_node(filename, type="file", x=xR, y = yR, label = label)
        
        # Add the filename to the file array
        fArr.append(filename)
        
    # If the degree is > windowSize-1 then add the term and edge
    # We use windowSize for in_degree counts, and windowSize*2 for degree counts
    #if int(degree)>((windowSize-1)*2):
    if int(degree)>(windowSize-1):
        # If we have not seen this term before, add a node to the graph
        if not term in tArr:
            # Create some random spatial positions
            xR2 = random.random()*1000
            yR2 = random.random()*1000

            # Add the term to the graph
            Gr.add_node(term, type="term", x=xR2, y=yR2)
            tArr.append(term)
            
        # Add the edge to the graph
        Gr.add_edge(filename, term, weight=degree)

for x in test_set:

    # Split out the 
    y = x.split("\t")
    filename = y[0]
    term = y[1]
    degree = y[2]
    label = filename[32:33]
    
    # If the filename is not already added, add it to the test Graph
    if not filename in fTest:

        # Add the file node to the graph
        GTest.add_node(filename, type="file", label = label)
        
        # Add the filename to the file array
        fTest.append(filename)
        
    # If the degree is > windowSize-1 then add the term and edge
    # We use windowSize for in_degree counts, and windowSize*2 for degree counts
    #if int(degree)>((windowSize-1)*2):
    if int(degree)>(windowSize-1):
        # If we have not seen this term before, add a node to the graph
        if not term in tTest:
            # Add the term to the graph
            GTest.add_node(term, type="term")
            tTest.append(term)
            
        # Add the edge to the graph
        GTest.add_edge(filename, term, weight=degree)

print("done in %0.3fs." % (time() - t0))


done in 2.119s.


In [12]:
# Remove terms with only one connection or > 95% fractile # of connections
t0 = time()
#d = Gr.in_degree()
d = Gr.degree()

# Store degrees to file
filename = 'C:\\BBC_Data\\bbc-sportext\\term_degree.csv'
myfile = open(filename, 'w')
wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

# List to allow calculation of percentile
degreeList = []

for t in tArr:
    myfile.write(t + "\t" + str(d[t]) + "\n")
    degreeList.append(d[t])


myfile.close()

    
print("List size: " + str(len(degreeList)))
perc = np.percentile(degreeList, 99.7)
print("Perc: " + str(perc))

for t in tArr:
    n = d[t]
    #print(n)
    if n < 2 or n > perc:
        # Remove Edge
        for f in Gr.neighbors(t):
            Gr.remove_edge(f,t)
        # Remove node
        Gr.remove_node(t)
        # Remove from tArr list
        tArr.remove(t)
        
print("done in %0.3fs." % (time() - t0))


List size: 3080
Perc: 107.815
done in 0.090s.


In [13]:
# Dictionaries for calculating Kullback-Leibler
Px = []
Qx = []
Py = []
Qy = []

# Load the Q arrays with the x/y probabilities
t0 = time()
for f in fArr:
    Qx.append(Gr.node[f]["x"])
    Qy.append(Gr.node[f]["y"])

print("done in %0.3fs." % (time() - t0))


done in 0.002s.


In [14]:

KLx = 999
KLy = 999

t0 = time()

# Run several itterations of moving points
#for i in range(0, 20):
i = 0
while KLx + KLy > 0.0000000125:
    
    # Reposition all the term nodes in the centroid of their documents
    for t in tArr:
        sumx=0
        sumy=0
        sumw=0

        for f in Gr.neighbors(t):
            w=float(Gr.edge[f][t]["weight"])
            sumw+=w
            sumx+=float(Gr.node[f]["x"])*w
            sumy+=float(Gr.node[f]["y"])*w

        if sumw > 0:
            Gr.node[t]["x"] = sumx/sumw
            Gr.node[t]["y"] = sumy/sumw
        else:
            print("0 weight on term " + t)

    StoreGraphToFile(Gr, i, 1, "Reposition Terms")
    
    pSumx = 0
    pSumy = 0
    
    # Reposition all the file nodes in the centroid of their terms
    for f in fArr:
        sumx=0
        sumy=0
        sumw=0

        for t in Gr.neighbors(f):
            w=float(Gr.edge[f][t]["weight"])
            sumw+=w
            sumx+=float(Gr.node[t]["x"])*w
            sumy+=float(Gr.node[t]["y"])*w

        if sumw > 0:
            Gr.node[f]["x"] = sumx/sumw
            pSumx += sumx/sumw
            Gr.node[f]["y"] = sumy/sumw
            pSumy += sumy/sumw
        else:
            print("0 weight on file " + f)

    # Load the P arrays with the x/y probabilities
    # and calculate the KL sum
    for f in fArr:
        Px.append(Gr.node[f]["x"])
        Py.append(Gr.node[f]["y"])
        #print("a" if x < y else "b")
        #KLx += ((Px[f] * math.log(Px[f]/Qx[f]))+(Qx[f] * math.log(Qx[f]/Px[f]))) if Px[f] > 0 else 0
        #KLy += ((Py[f] * math.log(Py[f]/Qy[f]))+(Qy[f] * math.log(Qy[f]/Py[f]))) if Py[f] > 0 else 0

    KLx = scipy.stats.entropy(Px, Qx)
    KLy = scipy.stats.entropy(Py, Qy)

    StoreGraphToFile(Gr, i, 2, "Reposition Files")
    
    # Assign the P sets to the Q sets for the next run
    Qx = Px
    Qy = Py
    Px = []
    Py = []
    
    print(str(i) + ": " + str(KLx) + ", " + str(KLy))
    i+=1
    
print("done in %0.3fs." % (time() - t0))


0: 0.240785566422, 0.228771574523
1: 0.0036341520349, 0.00344722509395
2: 0.000366987403635, 0.000340710369009
3: 6.22740630181e-05, 5.44596024961e-05
4: 1.41024392476e-05, 1.14860752237e-05
5: 3.89575463025e-06, 2.95207900108e-06
6: 1.24465848125e-06, 8.81089344457e-07
7: 4.41193681903e-07, 2.9437060077e-07
8: 1.68017419056e-07, 1.06986519152e-07
9: 6.72044535995e-08, 4.13827793886e-08
10: 2.78210649765e-08, 1.67573393302e-08
11: 1.18110090831e-08, 7.0175434077e-09
12: 5.11254276848e-09, 3.01242807218e-09
done in 2.452s.


In [15]:
# Extract values for Tree from Graph
Y = [];
Y_label = [];

for f in fArr:
    x = Gr.node[f]["x"]
    y = Gr.node[f]["y"]
    lab = Gr.node[f]["label"]
    Y.append([x,y])
    Y_label.append(lab)

tree = KDTree(Y)

In [42]:
for test_k in range(1,100):
    Actual = []
    Pred = []

    for f in fTest:
        sumx=0
        sumy=0
        sumw=0
        # What is the test set label?
        lab=GTest.node[f]["label"]
        pred_lab = dict()

        for t in GTest.neighbors(f):
            # is this test term in our term array?
            if t in tArr:
                # Get the weight of the edge from the test graph
                w=float(GTest.edge[f][t]["weight"])
                sumw+=w
                # Get the x,y of the term from the model graph
                sumx+=float(Gr.node[t]["x"])*w
                sumy+=float(Gr.node[t]["y"])*w

        if sumw > 0:
            # Get the centroid
            x = sumx/sumw
            y = sumy/sumw
            # Query the tree for nearest neighbor
            dist,ind = tree.query([[x,y]], k=test_k)
            for i in ind[0]:
                if Y_label[i] in pred_lab:
                    pred_lab[Y_label[i]] += 1
                else:
                    pred_lab[Y_label[i]] = 1

        else:
            pred_lab["UNK"] = 1

        #print(f + ": " + lab + ", pred=" + str(sorted(pred_lab.items(), key=operator.itemgetter(1), reverse=True)[0][0]))
        Actual.append(lab)
        Pred.append(str(sorted(pred_lab.items(), key=operator.itemgetter(1), reverse=True)[0][0]))

    precision = precision_score(Actual, Pred, average='weighted') 
    recall = recall_score(Actual, Pred, average='weighted')
    f1 = f1_score(Actual, Pred, average='weighted') 
    print(str(test_k) + ',' + str(precision) + ',' + str(recall) + ',' + str(f1))

1,0.778458372903,0.777777777778,0.767685768855
2,0.81899172626,0.79012345679,0.767890741277
3,0.84988423592,0.851851851852,0.850119791406
4,0.821214825769,0.827160493827,0.821082383071
5,0.815789224729,0.814814814815,0.814097616559
6,0.815801695112,0.827160493827,0.819486989078
7,0.796722009366,0.802469135802,0.79934403911
8,0.83567251462,0.83950617284,0.836611947723
9,0.814814814815,0.814814814815,0.814814814815
10,0.81794455274,0.814814814815,0.815653072149
11,0.826279017354,0.827160493827,0.825040397631
12,0.819590643275,0.827160493827,0.822195903027
13,0.821623643846,0.827160493827,0.822620087895
14,0.807793927845,0.814814814815,0.810406398411
15,0.839096464021,0.83950617284,0.838512729695
16,0.811804201863,0.814814814815,0.812581458558
17,0.805367494137,0.814814814815,0.808349026993
18,0.80548696845,0.814814814815,0.809067688378
19,0.8109739369,0.814814814815,0.811991664986
20,0.821149748126,0.827160493827,0.822797541442
21,0.821149748126,0.827160493827,0.822797541442
22,0.8075695

In [40]:
#cm = confusion_matrix(Actual, Pred)
#cm



In [122]:
##### CLUSTERING


# Change text label to number
def strToNum(str):
    rVal = 0
    if str == "B":
        rVal = 0
    elif str == "E":
        rVal = 1
    elif str == "P":
        rVal = 2
    elif str == "S":
        rVal = 3
    elif str == "T":
        rVal = 4
        
    return rVal


# Load the data from the graph into a dataframe
df = pd.DataFrame(columns=['filename','x','y','Label'])
df.Label = df.Label.astype(int)

for f in fArr:
    s = strToNum(f[32:33])
    df = df.append({"filename": f, "x":Gr.node[f]["x"], "y":Gr.node[f]["y"], "Label": s}, ignore_index=True)

#df = df.append({"filename": "file1", "x":1, "y":2}, ignore_index=True)
#df = df.append({"filename": "file2", "x":2, "y":3}, ignore_index=True)

In [123]:
# Cluster the documents using kmeans
kmeans = KMeans(n_clusters=5,init='k-means++')
result = kmeans.fit_predict(df[["x","y"]])

In [124]:
df["Cluster"]=result

4

In [127]:


cm = confusion_matrix(df.Label, df.Cluster)

In [128]:
cm

array([[213,   0,   0,   2, 295],
       [ 79,   2,   0, 293,  12],
       [402,   0,   1,   4,  10],
       [ 47,   1, 457,   6,   0],
       [ 81,   0,   0,  12, 308]])

In [151]:

tc = pd.DataFrame(columns=['term','weight','Cluster'])
tc.Cluster = tc.Cluster.astype(int)

for f in fArr:
    dftemp=df[df['filename'] == f]
    c = dftemp.get_value(dftemp.index[0], col='Cluster')
    for t in Gr.neighbors(f):
        w=float(Gr.edge[f][t]["weight"])
        tc = tc.append({"term": t, "weight": w, "Cluster": c}, ignore_index=True)
        
        

In [153]:
tc.to_csv("c:\\BBC_Data\\tc_out.csv")